## About this demo
It's a demo for LLM video/image generating training preprocessing.

Based on BMF, it's flexible to build and integrate algorithms into whole pipeline of preprocessing.

## 1. Environmental preparation

### 1.1 FFmpeg
FFmpeg 4.x or 5.x is needed by BMF when transcoding, check versions via apt:

In [ ]:
! apt show ffmpeg | grep "^Package:\|^Version:"

If the version meets the requirements, install ffmpeg via apt:

In [ ]:
! apt install -y ffmpeg

Otherwise, you need to compile ffmpeg from source, you can use the script we provided(only linux and macos now):

In [ ]:
! git clone https://github.com/BabitMF/bmf bmf
! ./bmf/scripts/build_ffmpeg.sh nasm yasm x264 x265

### 1.2 BMF
BMF can be installed in many ways, we use pip here:

In [ ]:
! pip3 install BabitMF

### 1.3 pip requirements

In [ ]:
! pip3 install easyocr pydantic onnxruntime

### 1.4 wurlitzer(optional)
This package is installed to show the BMF C++ logs in the colab console, otherwise only python logs are printed. This step is not necessary if you're not in a Colab or iPython notebook environment.

In [ ]:
!pip install wurlitzer
%load_ext wurlitzer

## 2. preprocessing demo

Download the video file we will be using first:

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1l8bDSrWn6643aDhyaocVStXdoUbVC3o2/view?usp=sharing -O big_bunny_10s_30fps.mp4

In [ ]:
! ffprobe big_bunny_10s_30fps.mp4

### 2.1 python code and download resources

In [ ]:
! git clone https://github.com/BabitMF/bmf.git

In [ ]:
%cd /content/bmf/bmf/demo/LLM_video_preprocessing
! pwd


In [ ]:
! wget https://github.com/BabitMF/bmf/releases/download/files/models.tar.gz && tar zxvf models.tar.gz -C ../../

In [ ]:
! ls -lrht ../../models

### 2.3 run demo

In [ ]:
import logging
from media_info import MediaInfo
from split import get_split_info
import torch
from clip_process import ClipProcess
import argparse
import os

logger = logging.getLogger('main')
logger.setLevel(logging.INFO)

scene_thres = 0.3

def get_timeline_list(pts_list, last):
    current = 0
    timelines = []
    for pts in pts_list:
        pts = pts/1000000
        if pts > current:
            timelines.append([current,pts])
        current = pts
    # last
    if last > current:
        timelines.append([current,last])
    return timelines
def video_processing_demo(input_file, mode, config):
    media_info = MediaInfo("ffprobe", input_file)
    duration = media_info.video_duration()
    logger.info(f"duration:{duration}")

    pts_list = get_split_info(input_file, scene_thres)
    timelines = get_timeline_list(pts_list, duration)
    logger.info(f"timelines:{timelines}")
    cp = ClipProcess(input_file, timelines, mode, config)
    cp.process()

def demo_run(input_file):
    model_path = "../../models/aes_transonnx_update3.onnx"

    torch.set_num_threads(4)
    mode = "ocr_crop,aesmod_module"
    config = {"output_configs":[{"res":"orig", "type":"jpg"}, {"res":"480", "type":"mp4"}]}
    video_processing_demo(input_file, mode, config)



In [ ]:
!ls /content

In [ ]:
demo_run("/content/big_bunny_10s_30fps.mp4")

In [ ]:
! cd clip_output && ls -lrth && cat clip_0_ocr_crop_result.json && cat clip_0_aesmod_module_result.json

### 2.3 visulize output video

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 800):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

# input video
show_video('clip_output/clip_1_480.mp4')